In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import sys  
import datetime


sys.path.append('..')
from utils.tte import tte


DATA_DIR = Path("../data/cl/processed") 

rolled_raw = pd.read_csv(DATA_DIR / "cl_front_month_raw.csv")
panama_rolled = pd.read_csv(DATA_DIR / "cl_front_month_panama_adjusted.csv")
proportional_rolled = pd.read_csv(DATA_DIR / "cl_front_month_proportional_adjusted.csv")

spreads = pd.read_csv(DATA_DIR / "cl_spreads.csv")


In [ ]:
raw = pd.read_csv("../data/cl/processed/cl_raw.csv", index_col=0, parse_dates=True)
raw[["letters", "number"]] = raw["symbol"].str.extract(r"([a-zA-Z]+)([0-9]+)")
raw["number"] = raw["number"].astype(int)
raw = raw.sort_values(by=["ts_event", "number", "letters"]).reset_index()
raw["month"] = raw["letters"].str[-1]
raw = raw.drop(columns=["letters"])

In [ ]:
raw[["ts_event", "symbol", "volume", "month", "number"]]

In [ ]:
raw["expiration_year"] = np.where((raw["ts_event"].dt.year//10 *10)+raw["number"]<raw["ts_event"].dt.year,((raw["ts_event"].dt.year//10)+1)*10,(raw["ts_event"].dt.year//10)*10) + raw["number"] #type: ignore

In [ ]:
from datetime import timedelta
from pandas.tseries.offsets import CustomBusinessDay
month_codes = "FGHJKMNQUVXZ"

conversions = {
    "F": "-12-25",
    "G": "-01-25",
    "H": "-02-25",
    "J": "-03-25",
    "K": "-04-25",
    "M": "-05-25",
    "N": "-06-25",
    "Q": "-07-25",
    "U": "-08-25",
    "V": "-09-25",
    "X": "-10-25",
    "Z": "-11-25"
}

#pattern = rf'^CL[{month_codes}]\d$'

raw["expiration_date"] = pd.to_datetime((raw["expiration_year"].astype(str) + raw["month"].map(conversions))) #type: ignore
raw["expiration_date"] = raw["expiration_date"].dt.tz_localize("UTC") #type: ignore
raw["expiration_date"] = raw["expiration_date"] - pd.offsets.BusinessDay(3)
raw.loc[raw["month"] == "F", "expiration_date"] = pd.to_datetime((raw["expiration_date"].loc[raw["month"] == "F"].dt.year.astype(int) - 1).astype(str) + "-12-25").dt.tz_localize("UTC") - pd.offsets.BusinessDay(3)  #type: ignore

raw


In [ ]:
raw["tte"] = (raw["expiration_date"] - raw["ts_event"]).dt.days / 365.25 #type: ignore

In [ ]:
#raw = raw.drop(columns=["number", "month", "expiration_year"])
spreads[["ts_event", "symbol", "close", "volume"]]


In [ ]:
first_month = rf'^CL[{month_codes}]\d$'
second_month = rf'^CL[{month_codes}]\d$'


#spreads[["letters"]] = spreads["symbol"].str.extract() #type: ignore
spreads[["month_from", "year_from", "month_to", "year_to"]] = spreads["symbol"].str.extract(rf"CL([{month_codes}])([0-9])-CL([{month_codes}])([0-9])")
spreads = spreads[["ts_event", "symbol", "close", "volume", "month_from", "year_from", "month_to", "year_to"]]
spreads["year_from"] = spreads["year_from"].astype(int)
spreads["year_to"] = spreads["year_to"].astype(int)
spreads["ts_event"] = pd.to_datetime(spreads["ts_event"])

In [ ]:

#spreads["from_date"] = pd.to_datetime(((np.where((spreads["ts_event"].dt.year//10 *10)+spreads["year_from"]<spreads["ts_event"].dt.year,2020,2010) + spreads["year_from"]).astype(str)+ spreads["month_from"].map(conversions))).dt.tz_localize('UTC') #type: ignore
#spreads["to_date"] = pd.to_datetime(((np.where((spreads["ts_event"].dt.year//10 *10)+spreads["year_to"]<spreads["ts_event"].dt.year,2020,2010) + spreads["year_to"]).astype(str)+ spreads["month_to"].map(conversions))).dt.tz_localize('UTC') #type: ignore
#
#spreads["year"] = spreads["ts_event"].dt.year//10 * 10 #type: ignore
#spreads["number"] = spreads["year_from"]
#spreads["test_year"] = spreads["year"] + spreads["number"]
#spreads["test2"] = np.where(spreads["test_year"]<spreads["ts_event"].dt.year, spreads["test_year"]+10, spreads["test_year"]) #type: ignore
#spreads[["ts_event", "tte", "year", "year_from", "test_year", "test2", "from_date"]]


In [ ]:


spreads["from_date"] = pd.to_datetime(((np.where((spreads["ts_event"].dt.year//10 *10)+spreads["year_from"]<spreads["ts_event"].dt.year,((spreads["ts_event"].dt.year//10)+1)*10,(spreads["ts_event"].dt.year//10)*10) + spreads["year_from"]).astype(str)+ spreads["month_from"].map(conversions))).dt.tz_localize('UTC') #type: ignore
spreads["to_date"] = pd.to_datetime(((np.where((spreads["ts_event"].dt.year//10 *10)+spreads["year_to"]<spreads["ts_event"].dt.year,((spreads["ts_event"].dt.year//10)+1)*10,(spreads["ts_event"].dt.year//10)*10) + spreads["year_to"]).astype(str)+ spreads["month_to"].map(conversions))).dt.tz_localize('UTC') #type: ignore

spreads["from_date"] = spreads["from_date"] - pd.offsets.BusinessDay(3)
spreads["to_date"] = spreads["to_date"] - pd.offsets.BusinessDay(3)


spreads.loc[spreads["month_from"] == "F", "from_date"] = pd.to_datetime((spreads["from_date"].loc[spreads["month_from"] == "F"].dt.year.astype(int) - 1).astype(str) + "-12-25").dt.tz_localize("UTC") - pd.offsets.BusinessDay(3)  #type: ignore
spreads.loc[spreads["month_to"] == "F", "to_date"] = pd.to_datetime((spreads["to_date"].loc[spreads["month_to"] == "F"].dt.year.astype(int) - 1).astype(str) + "-12-25").dt.tz_localize("UTC") - pd.offsets.BusinessDay(3)  #type: ignore

spreads["tenor"] = (spreads["to_date"] - spreads["from_date"]).dt.days / 365.25 #type: ignore
spreads["tte"] = (spreads["to_date"] - spreads["ts_event"]).dt.days / 365.25 #type: ignore

spreads.sort_values(by=["ts_event", "year_from", "month_from", "tenor"], inplace=True)

spreads.reset_index(drop=True, inplace=True)
spreads


In [ ]:
raw = raw[raw["ts_event"] >= pd.to_datetime("2011-11-27").tz_localize("UTC")].reset_index(drop=True)
raw[["ts_event", "symbol", "close", "volume", "month", "expiration_date", "tte"]]

In [ ]:
df = raw.loc[raw["ts_event"] >= pd.to_datetime("2011-11-27").tz_localize("UTC")]
df["month_to"] = df["month"]
df["year_to"] = df["number"]
df = df.rename(columns={"month": "month_from", "number": "year_from"})
df = df[["ts_event", "symbol", "close", "volume", "month_from",  "year_from", "month_to", "year_to", "tte"]]
df


In [ ]:
dfs = spreads.loc[spreads["ts_event"] >= pd.to_datetime("2011-11-27").tz_localize("UTC")]
dfs = dfs[["ts_event", "symbol", "close", "volume", "month_from", "year_from", "month_to", "year_to", "tte"]]
#dfs = dfs.rename(columns={"month_to": "month", "year_to": "number"})
dfs

In [ ]:
target = pd.concat([df, dfs], ignore_index=True)
target.sort_values(by=["tte", "volume"], inplace=True, ascending=[True, False])
target.reset_index(drop=True, inplace=True)
target = target.groupby(["ts_event", "tte"]).first().reset_index()
#target[["ts_event", "symbol", "close", "volume", "tte"]]
target

In [ ]:
#target = target[target["ts_event"] == pd.to_datetime("2017-11-08").tz_localize("UTC")]

target["year_from"] =  np.where(((target["ts_event"].dt.year//10 * 10)+target["year_from"])<target["ts_event"].dt.year, target["year_from"]+10, target["year_from"]) #type: ignore
target["year_to"] =  np.where(((target["ts_event"].dt.year//10 * 10)+target["year_to"])<target["ts_event"].dt.year, target["year_to"]+10, target["year_to"]) #type: ignore

target

In [ ]:
m_map = {'F':1, 'G':2, 'H':3, 'J':4, 'K':5, 'M':6, 'N':7, 'Q':8, 'U':9, 'V':10, 'X':11, 'Z':12}
#target["month_from_num"] = target["month_from"].map(m_map) 
#target["month_to_num"] = target["month_to"].map(m_map).fillna(0).astype(int) + target["year_to"]*12
#
#target["abs_from"] = target["month_from_num"] + target["year_from"]*12
#target["abs_to"] = target["month_to_num"] + target["year_to"]*12
#
#start_month = target["abs_from"].min()
#target["idx_from"] = target["abs_from"] - start_month
#target["idx_to"] = target["abs_to"] - start_month
target["position_from"] = target["month_from"].map(m_map) + target["year_from"]*12
target["position_to"] = target["month_to"].map(m_map) + target["year_to"]*12
start_month = target["position_from"].min()
target["position_from"] = target["position_from"] - start_month
target["position_to"] = target["position_to"] - start_month

target[target["ts_event"] == pd.to_datetime("2020-11-08").tz_localize("UTC")]


In [ ]:
row_indices = np.arange(len(target))
A = np.zeros((len(target), target["position_to"].max()+1))
A[row_indices, target["position_to"]] = -1
A[row_indices, target["position_from"]] = 1



In [ ]:
x, residuals, rank, s = np.linalg.lstsq(A, target["close"], rcond=None)
result = pd.DataFrame()
result["date"] = target["ts_event"]
result["symbol"] = "CL" + target["month_to"] + (target["year_to"] % 10).astype(str)
result["close"] = np.round(x[target["position_to"]], 3)
result["volume"] = target["volume"]
result["tte"] = target["tte"]
result

In [ ]:
result = result[result["tte"] <= 2]

In [ ]:
available = []
for date, row in result.groupby("date"):
    available.append((date, len(row)))
print(f"there are on average {np.mean([a[1] for a in available]):.2f} contracts available per date")

In [ ]:
result[result["date"] == pd.to_datetime("2020-11-08").tz_localize("UTC")]

In [ ]:
result.reset_index(drop=True, inplace=True)
result.to_csv(DATA_DIR / "cl_forward_curve_reconstructed.csv", index=False)

In [ ]:
slice = result[result["date"] == pd.to_datetime("2020-11-08").tz_localize("UTC")]
plt.plot(slice["tte"], slice["close"], label='Raw Forward Curve', marker='o')
plt.title('WTI Crude Forward Curve on 2017-11-08')
plt.xlabel('Time to Expiration (Years)')
plt.ylabel('Price (USD)')
fig = plt.gcf()
fig.set_size_inches(12, 6)
fig.set_dpi(600)
plt.tight_layout()
plt.legend()
plt.show()


In [ ]:
from matplotlib.pylab import beta


def nelson_siegel_svensson(tau, beta0, beta1, beta2, beta3, lambda1, lambda2):
    term1 = (1-np.exp(-tau / lambda1))/(tau/lambda1)
    term2 = term1 - np.exp(-tau / lambda1)
    term3 = (1-np.exp(-tau / lambda2))/(tau/lambda2) - np.exp(-tau / lambda2)
    return beta0 + beta1 * term1 + beta2 * term2 + beta3 * term3

def nelson_siegel(tau, beta0, beta1, beta2, lambda_):
    term1 = (1 - np.exp(-tau / lambda_)) / (tau / lambda_)
    term2 = term1 - np.exp(-tau / lambda_)
    return beta0 + beta1 * term1 + beta2 * term2

def objective_s(x, tau, close):
    beta0, beta1, beta2, beta3, lambda1, lambda2 = x
    prices = nelson_siegel_svensson(tau, beta0, beta1, beta2, beta3, lambda1, lambda2)
    loss = np.sum((prices - close) ** 2)
    return loss

def objective(x, tau, close):
    beta0, beta1, beta2, lambda_ = x
    prices = nelson_siegel(tau, beta0, beta1, beta2, lambda_)
    loss = np.sum((prices - close) ** 2)
    return loss

def initial_guess_ns(tau, close):
    beta0 = np.mean(close[-3:]) if len(close) >=3 else np.mean(close)
    beta1 = close[0] - beta0
    if len(close) >=2:
        middle_idx = len(close)//2
        expected_middle = beta0 + beta1 * ((1 - np.exp(-tau[middle_idx]/1.0)) / (tau[middle_idx]/1.0))
        beta2 = close[middle_idx] - expected_middle
    else:
        beta2 = 0.0
    
    lambda_ = np.percentile(tau, 40)
    return [beta0, beta1, beta2, lambda_]

def initial_guess_nss(tau, close):
    beta0 = np.mean(close[-3:]) if len(close) >=3 else np.mean(close)
    beta1 = close[0] - beta0
    if len(close) >=2:
        middle_idx = len(close)//2
        expected_middle = beta0 + beta1 * ((1 - np.exp(-tau[middle_idx]/1.0)) / (tau[middle_idx]/1.0))
        beta2 = close[middle_idx] - expected_middle
    else:
        beta2 = 0.0

    if len(close) >= 4:
        
        idx_3q = (3 * len(close)) // 4
        m_3q = tau[0][idx_3q]
        
    
        expected_3q = (beta0 + (beta1 + beta2) * 
                      ((1 - np.exp(-m_3q / 1.0)) / (m_3q / 1.0)))    
        nss_factor = ((1 - np.exp(-m_3q / 1.0)) / (m_3q / 1.0) - 
                      np.exp(-m_3q / 1.0))
        if abs(nss_factor) > 1e-6:
            beta3 = (close[idx_3q] - expected_3q) / nss_factor
        else:
            beta3 = 0.0
    else:
        beta3 = 0.0

    lambda1 = np.percentile(tau, 40)
    lambda2 = np.percentile(tau, 70)
    return [beta0, beta1, beta2, beta3, lambda1, lambda2]

In [ ]:
from scipy.optimize import minimize
from joblib import Parallel, delayed

bounds_ns = [(10, 150), (-100, 100), (-100, 100), (0.01, 50)]

def solve_ns_row(date, row):
    tau = row["tte"].values
    close = row["close"].values
    x0 = initial_guess_ns(tau, close)
    #x0 = [row["close"].iloc[-1], row["close"].iloc[0]-row["close"].iloc[-1], 5, 0.5]
    
    res = minimize(objective, x0, args=(tau, close), method='L-BFGS-B', bounds=bounds_ns)
    beta0, beta1, beta2, lambda_ = res.x
    return {"date": date, "beta0": beta0, "beta1": beta1, "beta2": beta2, "lambda": lambda_}

# Fit Nelson-Siegel model
temp_results = Parallel(n_jobs=-1)(delayed(solve_ns_row)(date, row) for date, row in result.groupby("date"))
params_ns = pd.DataFrame(temp_results)
params_ns.to_csv(DATA_DIR / "forward_curve_ns_params.csv", index=False)

# Merge and calculate metrics
final_ns = pd.merge(result, params_ns, on='date', how='left')
final_ns["nelson_siegel"] = nelson_siegel(final_ns["tte"], final_ns["beta0"], final_ns["beta1"], final_ns["beta2"], final_ns["lambda"])
final_ns["loss"] = (final_ns["close"] - final_ns["nelson_siegel"])**2

mae_ns = np.abs(final_ns["close"] - final_ns["nelson_siegel"]).mean()
rmse_ns = np.sqrt(final_ns["loss"].mean())
print(f"Root Mean Squared Error of Nelson-Siegel Fit: {rmse_ns:.4f}")
print(f"Mean Absolute Error of Nelson-Siegel Fit: {mae_ns:.4f}")
mean_ns = final_ns["close"].mean()
print(f"This is off by {rmse_ns/mean_ns:.4%} of the average price.")
RSS_ns = final_ns["loss"].sum()
final_ns["mean"] = final_ns["close"].groupby(final_ns["date"]).transform('mean')
TSS_ns = np.sum((final_ns["close"] - final_ns["mean"])**2)
R_squared_ns = 1 - RSS_ns/TSS_ns
print(f"R-squared of Nelson-Siegel Fit: {R_squared_ns:.4f}")

In [ ]:
residuals = final_ns["close"] - final_ns["nelson_siegel"]
plt.scatter(final_ns["tte"], residuals, alpha=0.5, s=1)
plt.axhline(0, color='red', linestyle='--')

In [ ]:
bounds_nss = [(10, 150), (-100, 100), (-100, 100), (-100, 100), (0.01, 50), (0.01, 50)]

def solve_nss_row(date, row):
    tau = row["tte"].values
    close = row["close"].values
    x0 = [row["close"].iloc[-1], row["close"].iloc[0]-row["close"].iloc[-1], 5, -2, 0.5, 2.0]
    
    res = minimize(objective_s, x0, args=(tau, close), method='L-BFGS-B', bounds=bounds_nss)
    beta0, beta1, beta2, beta3, lambda1, lambda2 = res.x
    return {"date": date, "beta0": beta0, "beta1": beta1, "beta2": beta2, "beta3": beta3, "lambda1": lambda1, "lambda2": lambda2}

# Fit Nelson-Siegel-Svensson model
temp_results_nss = Parallel(n_jobs=-1)(delayed(solve_nss_row)(date, row) for date, row in result.groupby("date"))
params_nss = pd.DataFrame(temp_results_nss)
params_nss.to_csv(DATA_DIR / "forward_curve_nss_params.csv", index=False)

# Merge and calculate metrics
final_nss = pd.merge(result, params_nss, on='date', how='left')
final_nss["nelson_siegel_svensson"] = nelson_siegel_svensson(final_nss["tte"], final_nss["beta0"], final_nss["beta1"], final_nss["beta2"], final_nss["beta3"], final_nss["lambda1"], final_nss["lambda2"])
final_nss["loss"] = (final_nss["close"] - final_nss["nelson_siegel_svensson"])**2

mae_nss = np.abs(final_nss["close"] - final_nss["nelson_siegel_svensson"]).mean()
rmse_nss = np.sqrt(final_nss["loss"].mean())
print(f"Root Mean Squared Error of Nelson-Siegel-Svensson Fit: {rmse_nss:.4f}")
print(f"Mean Absolute Error of Nelson-Siegel-Svensson Fit: {mae_nss:.4f}")
mean_nss = final_nss["close"].mean()
print(f"This is off by {rmse_nss/mean_nss:.4%} of the average price.")
RSS_nss = final_nss["loss"].sum()
final_nss["mean"] = final_nss["close"].groupby(final_nss["date"]).transform('mean')
TSS_nss = np.sum((final_nss["close"] - final_nss["mean"])**2)
R_squared_nss = 1 - RSS_nss/TSS_nss
print(f"R-squared of Nelson-Siegel-Svensson Fit: {R_squared_nss:.4f}")

In [ ]:
residuals = final_nss["close"] - final_nss["nelson_siegel_svensson"]
plt.scatter(final_nss["tte"], residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')